In [ ]:
import re
import pickle
import random
import os
import gc
import csv
import sys
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import dstack
use_gpu=True
import keras
import matplotlib.cm as cm
import seaborn as sns
from keras.layers import Lambda
from keras import layers
from operator import add
from ast import literal_eval
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')
from collections import Counter
from nltk.tokenize import WordPunctTokenizer
from nltk import TweetTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.layers import InputSpec, Layer
from tensorflow.keras.optimizers import Adam
#from keras.layers.core import Lambda
from keras import backend as K
from keras.layers import Layer
from keras.utils import plot_model
from keras.models import Model, Sequential, load_model, model_from_json
from keras.layers import Dense, Dropout, Activation, SpatialDropout1D, GaussianNoise, GaussianDropout
from keras.layers import Input, Embedding, Flatten, Conv1D, Conv2D, Reshape, Concatenate, concatenate
from keras.layers import LeakyReLU, LSTM, GRU, Bidirectional, BatchNormalization, TimeDistributed, Add
from keras.layers import GlobalMaxPooling1D, MaxPooling1D, MaxPool2D, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, Callback, EarlyStopping
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, f1_score, classification_report, accuracy_score, confusion_matrix
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve, auc
from keras.layers import Layer
from keras.utils import to_categorical
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score, classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score
from tensorflow.keras.regularizers import l2

In [ ]:
path = '...'
FREQ_DIST_FILE = path+'/processed_freqdist.pkl'
BI_FREQ_DIST_FILE = path+'/processed_freqdist-bi.pkl'
TRAIN_PROCESSED_FILE =  path+'/processed_train.csv'
TEST_PROCESSED_FILE = path+'/processed_test.csv'
processed_data_csv = path+'/processed_dataset.csv'
cols = ['abstract','category']
df = pd.read_csv(processed_data_csv, header = None, names = cols, encoding = 'utf-8')
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
x = df.abstract
y = df.category
num_classes = 3
SEED = 200
max_length = 150
df.drop_duplicates(inplace=True)
duplicate_rows = df[df.duplicated()]

Toplam tekrarlanan satır sayısı: 0


In [ ]:
def top_n_words(pkl_file_name, N, shift=0):
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    words = {p[0]: i + shift for i, p in enumerate(most_common)}
    return words

In [ ]:
def process_abstracts(csv_file, test_file):
    abstracts = []
    labels = []
    with open(csv_file, 'r', encoding = "utf-8") as csv:
        lines = csv.readlines()
        total = len(lines)
        category_names = []
        for i, line in enumerate(lines):
            abstract = line[:line.find(',')].strip()
            category = line[1 + line.find(','):].strip().strip('\"').strip('\n').strip('\;')
            abstracts.append(abstract)
            labels.append(category)
            if category not in category_names:
                category_names.append(category)
    return abstracts, labels, category_names

In [ ]:
class AttentionWithContext(Layer):
    def __init__(self, **kwargs):
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name="W_q", shape=(input_shape[-1], input_shape[-1]), initializer="glorot_uniform", trainable=True)
        self.W_k = self.add_weight(name="W_k", shape=(input_shape[-1], input_shape[-1]), initializer="glorot_uniform", trainable=True)
        self.W_v = self.add_weight(name="W_v", shape=(input_shape[-1], input_shape[-1]), initializer="glorot_uniform", trainable=True)
        super(AttentionWithContext, self).build(input_shape)

    def call(self, x):
        q = tf.matmul(x, self.W_q)
        k = tf.matmul(x, self.W_k)
        v = tf.matmul(x, self.W_v)

        attn_scores = tf.matmul(q, k, transpose_b=True)
        attn_scores = tf.nn.softmax(attn_scores)

        output = tf.matmul(attn_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape


In [ ]:
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.2, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [ ]:
### tokenize data using bert tokenizer
from transformers import *
#Get BertTokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = BertTokenizer.from_pretrained(model_name)
vocab_size = len(tokenizer.get_vocab())
model = BertModel.from_pretrained(model_name)

In [ ]:
embedding_matrix = model.embeddings.word_embeddings.weight

In [ ]:
numpy_embedding_matrix = embedding_matrix.detach().numpy()

In [ ]:
embedding_dim = model.embeddings.word_embeddings.weight.size(1)

In [ ]:
tokenized_x_train = [tokenizer.tokenize(com) for com in x_train]
tokenized_x_train = [sent[:max_length] for sent in tokenized_x_train]

In [ ]:
tokenized_x_validation = [tokenizer.tokenize(com) for com in x_validation]
tokenized_x_validation = [sent[:max_length] for sent in tokenized_x_validation]

In [ ]:
tokenized_x_test = [tokenizer.tokenize(com) for com in x_test]
tokenized_x_test = [sent[:max_length] for sent in tokenized_x_test]

In [ ]:
for i in range(len(tokenized_x_train)):
    sent = tokenized_x_train[i]
    sent = ['[CLS]'] + sent + ['[SEP]']
    tokenized_x_train[i] = sent
#Convert tokens into IDs
input_ids_x_train = [tokenizer.convert_tokens_to_ids(com) for com in tokenized_x_train]
input_ids_x_train = tf.keras.preprocessing.sequence.pad_sequences(input_ids_x_train, maxlen=max_length+2, truncating='post', padding='post')

In [ ]:
for i in range(len(tokenized_x_validation)):
    sent = tokenized_x_validation[i]
    sent = ['[CLS]'] + sent + ['[SEP]']
    tokenized_x_validation[i] = sent
input_ids_x_validation = [tokenizer.convert_tokens_to_ids(com) for com in tokenized_x_validation]
#Pad our tokens which might be less than max_length size
input_ids_x_validation = tf.keras.preprocessing.sequence.pad_sequences(input_ids_x_validation, maxlen=max_length+2, truncating='post', padding='post')

In [ ]:
for i in range(len(tokenized_x_test)):
    sent = tokenized_x_test[i]
    sent = ['[CLS]'] + sent + ['[SEP]']
    tokenized_x_test[i] = sent
input_id_x_test = [tokenizer.convert_tokens_to_ids(com) for com in tokenized_x_test]
#Pad our tokens which might be less than max_length size
input_ids_x_test = tf.keras.preprocessing.sequence.pad_sequences(input_id_x_test, maxlen=max_length+2, truncating='post', padding='post')

In [ ]:
y_train              = np.array(y_train)
y_validation         = np.array(y_validation)
y_test               = np.array(y_test)
label_encoder = LabelEncoder()
y_train              = label_encoder.fit_transform(y_train)
y_validation         = label_encoder.transform(y_validation)
y_test               = label_encoder.transform(y_test)
#-------------------------
y_train              = to_categorical(y_train, num_classes=num_classes)
y_validation         = to_categorical(y_validation, num_classes=num_classes)
y_test               = to_categorical(y_test, num_classes=num_classes)

In [ ]:
#In[ ]---------------------MODEL------------------------------------
file_path = pathModels + "HybridGAN.hdf5"
check_point = ModelCheckpoint(file_path, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")
early_stop = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)
m_name = 'HybridGAN_'
inp = Input(shape=(max_length+2,), name=m_name+'inp')
x_gNoise=GaussianNoise(0.2)(inp)
#embd = Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length+2)(x_gNoise)
embd = Embedding(numpy_embedding_matrix.shape[0], embedding_dim, weights = [numpy_embedding_matrix], trainable = True)(x_gNoise)

x_LSTM= LSTM(64, return_sequences=True)(embd)
x_sDrp = SpatialDropout1D(0.2, name = 'spdrop_1')(x_LSTM)

x_biLSTM = Bidirectional(LSTM(units=64, return_sequences=True))(x_sDrp)
x_biLSTM_reshaped = Reshape((-1, 2*64))(x_biLSTM)
x_biLSTM_att = AttentionWithContext()(x_biLSTM_reshaped)
x_dropout1 = Dropout(0.2, name=m_name+'dropout_1')(x_biLSTM_att)

x_biGRU = Bidirectional(GRU(units=64, return_sequences=True))(x_dropout1)
x_dropout2 = Dropout(0.2, name=m_name+'dropout_2')(x_biGRU)
x_pool2 = GlobalMaxPooling1D(name=m_name+'pool_1')(x_dropout2)

dense1 = Dense(64, activation='softmax', name=m_name+'dense')(x_pool2)

output = Dense(num_classes,  name=m_name+'out')(dense1)

model = Model(inputs=inp, outputs=output)
optimizer = Adam(learning_rate=1e-3)
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
#print(model.summary())

In [ ]:

history = model.fit(input_ids_x_train, y_train, batch_size=128, epochs=6, validation_data=(input_ids_x_validation, y_validation),
                    verbose=1, callbacks=[check_point, early_stop])
#In[ ]---------------------/MODEL------------------------------------

Epoch 1/6
335/335 [==============================] - ETA: 0s - loss: 6.5257 - accuracy: 0.1135
Epoch 1: val_accuracy improved from -inf to 0.33920, saving model to /content/drive/MyDrive/doktora/PROJECT/RADIOLOGY/BertTokenDL/HybridGenAIDetection.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


335/335 [==============================] - 294s 857ms/step - loss: 6.5257 - accuracy: 0.1135 - val_loss: 10.6177 - val_accuracy: 0.3392
Epoch 2/6
335/335 [==============================] - ETA: 0s - loss: 10.6780 - accuracy: 0.3356
Epoch 2: val_accuracy did not improve from 0.33920
335/335 [==============================] - 273s 815ms/step - loss: 10.6780 - accuracy: 0.3356 - val_loss: 10.6177 - val_accuracy: 0.3392
Epoch 3/6
335/335 [==============================] - ETA: 0s - loss: 10.6727 - accuracy: 0.3354
Epoch 3: val_accuracy did not improve from 0.33920
335/335 [==============================] - 270s 807ms/step - loss: 10.6727 - accuracy: 0.3354 - val_loss: 10.6177 - val_accuracy: 0.3392
Epoch 4/6
335/335 [==============================] - ETA: 0s - loss: 10.6775 - accuracy: 0.3356
Epoch 4: val_accuracy did not improve from 0.33920
335/335 [==============================] - 268s 800ms/step - loss: 10.6775 - accuracy: 0.3356 - val_loss: 10.6177 - val_accuracy: 0.3392
Epoch 5/6
33

In [ ]:
# Tüm eğitim ve doğrulama loss değerlerini alın
all_train_loss = history.history['loss']
all_val_loss = history.history['val_loss']

# Tüm epoch'ların perplexity değerlerini hesaplayın
all_train_perplexity = [2 ** loss for loss in all_train_loss]
all_val_perplexity = [2 ** loss for loss in all_val_loss]

# Ortalama perplexity değerlerini hesaplayın
avg_train_perplexity = sum(all_train_perplexity) / len(all_train_perplexity)
avg_val_perplexity = sum(all_val_perplexity) / len(all_val_perplexity)

# Elde ettiğiniz ortalama perplexity değerlerini kullanabilirsiniz
print("Ortalama Eğitim Perplexity -HybridGAN:", avg_train_perplexity)
print("Ortalama Doğrulama Perplexity -HybridGAN:", avg_val_perplexity)


In [ ]:
# Modeli test veri seti üzerinde değerlendirin
test_loss = model.evaluate(input_ids_x_test, y_test, verbose=0)[0]
# Test veri seti üzerindeki perplexity değerini hesaplayın
test_perplexity = 2 ** test_loss
# Elde ettiğiniz test perplexity değerini kullanabilirsiniz
print("Test Perplexity:", test_perplexity)

In [ ]:
loaded_mymodel= load_model(pathModels + 'HybridGAN.hdf5', custom_objects={'AttentionWithContext': AttentionWithContext , "tf": tf})
print('loaded_mymodel loaded')
yhat_mymodel = loaded_mymodel.predict(input_ids_x_test)
print('yhat_mymodel complete!')

In [ ]:
classifiers = [yhat_mymodel]
for c,i in zip(classifiers,range(len(classifiers))):
  y_pred = (c > 0.5).astype(np.int32)
  y_pred = y_pred.tolist()
y_pred = np.array(y_pred)

multilabel_conf_matrix = multilabel_confusion_matrix(y_pred, y_test)
print(multilabel_conf_matrix)

In [ ]:
category_names = ['Human', 'Paraphrased', 'GPT']
#----------------
print("HybridGAN")
for class_index, conf_matrix in enumerate(multilabel_conf_matrix):
    class_name = category_names[class_index]
    print(f"Confusion Matrix for Class {class_name}:\n\n", conf_matrix)
    TP, FP, FN, TN = conf_matrix.ravel()
    print(f'TP: {TP}, FP: {FP}, FN: {FN}, TN: {TN} \n')
    y_test_class = y_test[:, class_index]
    y_pred_class = y_pred[:, class_index]
    #conf_matrix = confusion_matrix(y_test_class, y_pred_class)
    acc = accuracy_score(y_test_class, y_pred_class)
    f1 = f1_score(y_test_class, y_pred_class)
    precision = precision_score(y_test_class, y_pred_class)
    recall = recall_score(y_test_class, y_pred_class)
    mcc = matthews_corrcoef(y_test_class, y_pred_class)
    kappa = cohen_kappa_score(y_test_class, y_pred_class)
    roc_curve = roc_auc_score(y_test_class, y_pred_class)
    accuracy_values = []
    metrics = []
#--------------------------------------------
    accuracy_values.append(round(acc, 2))
#--------------------------------------------
    metrics.append({
        "Class": class_name,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "roc_curve":roc_curve,
        "mcc":mcc,
        "kappa":kappa
    })
    print(f'acc - {round(acc, 2)}')
    print(f'f1 - {round(f1, 2)}')
    print(f'precision - {round(precision, 2)}')
    print(f'recall - { round(recall, 2)}')
    print(f'mcc - {round(mcc, 2)}')
    print(f'kappa - {round(kappa, 2)}')
    print(f'roc_curve - {round(roc_curve, 2)} \n')
#--------------------------------------------
    X_ = ["Accuracy","F1 Score", "Precision", "Recall", "MCC", "Kappa", "Roc Curve"]
    v_ = accuracy_values

    colors = [cm.inferno(i / 7) for i in range(7)]
    plt.figure(figsize=(4, 3))
    bars = plt.bar(X_, v_, color=colors, width=0.5)
    plt.bar(X_, v_, color=colors, width=0.5)
    plt.ylabel('Values')
    plt.title(class_name+"\n")
    plt.xticks(rotation=45)
    plt.ylim(0, 1)  # Y ekseni başlangıcı ve bitişi
    plt.yticks(np.arange(0, 1.1, 0.25))  # Y ekseni işaretleme aralığı

    # Barların üzerine değerleri ekleme
    for bar, value in zip(bars, v_):
      plt.text(bar.get_x() + bar.get_width() / 2 - 0.1, value + 0.02, round(value, 2), ha='center', va='bottom')

    plt.show()
    print('\n')

# Tüm sınıfların ağırlıklı ortalamasını hesaplayın
overall_accuracy = np.average(accuracy_values)

# Genel performans metriklerini ekrana yazdırın
print(f"Genel Doğruluk (Overall Accuracy): {round(overall_accuracy, 2)}")
